# Serving an LLM application as an endpoint using FastAPI
## Content
1. Example of API creation
2. Build End-to-End application in Python

# 1. Example of API creation

In [4]:
from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}

# 2. Build End-to-End application in Python

## 2.1. Simple function

In [ ]:
from openai import OpenAI
import os

os.environ['OPENAI_API_KEY'] = '...'

# Iniialize OpenAI Client with your API key
client = OpenAI()

def translate_text(input_str):
    completion = client.chat.completion.create(
        messages=[
            {
                'role': 'system',
                'content': 'You are an expert translator who translates text from english to french and only return translated text',
            },
            {
                'role': 'user',
                'content': 'input_str',
            }
        ]        
    )
    return completion.choices[0].message.content

In [ ]:
# test the function
translate_text('this is a test string to translate')

## 2.2. Endpoint API

In [ ]:
from fastapi import FastAPI
from fastapi import HTTPException
from pydantic import BaseModel
from openai import OpenAI
import os

os.environ['OPENAI_API_KEY'] = '...'

# Initialize OpenAI client with your API key
client = OpenAI()

# Initialize FastAPI client
app = FastAPI()

# Create class with pydantic BaseModel
class TranslationRequest(BaseModel):
    input_str: str

def translate_text(input_str):
    completion = client.chat.completions.create(
        model='gpt-5-0125-preview',
        messages=[
            {
                'role': 'system',
                'content': 'You are an expert translator who translates text from english to french and only return translated text'
            },
            {
                'role': 'user',
                'content': input_str
            }
        ]
    )
    return completion.choices[0].message.content

@app.post('/translate/') # This line decorates 'translate' as a POST endpoint
async def translate(request:TranslationRequest):
    try:
        # call your translation function
        translated_text = translate_text(request.input_str)
        return {'translated_text': translated_text}
    except Exception as e:
        # handle exceptions for error during translation
        raise HTTPException(status_code=500, detail=str(e))

Here, main.py refers to the file containing your code. The term app points to the FastAPI application instance, which is identified in your code app = FastAPI().

To run and test this API locally simply run the following command:

    uvicorn main:app --reload

Navigate to localhost:8000/docs to see a UI interface